In [18]:
!pip install modin[ray]

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 17.2 MB 144 kB/s  eta 0:00:01     |██████████████████████▏         | 11.9 MB 17.5 MB/s eta 0:00:01     |███████████████████████████▎    | 14.7 MB 17.5 MB/s eta 0:00:01     |█████████████████████████████▉  | 16.1 MB 17.5 MB/s eta 0:00:01
     |████████████████████████████████| 47.5 MB 131 kB/s  eta 0:00:01    |███                             | 4.4 MB 25.4 MB/s eta 0:00:02     |█████████▋                      | 14.3 MB 25.4 MB/s eta 0:00:02     |████████████▌                   | 18.5 MB 25.4 MB/s eta 0:00:02     |█████████████▍                  | 20.0 MB 25.4 MB/s eta 0:00:02     |██████████████▍                 | 21.4 MB 25.4 MB/s eta 0:00:02     |██████████████████              | 26.7 MB 11.2 MB/s eta 0:00:02     |█████████████████████▋          | 32.2 MB 11.2 MB/s eta 0:00:02     |██████████████████████▋         | 33.6 MB 11.2 MB/s eta 0:00:02     |████████████████████

     |████████████████████████████████| 100 kB 542 kB/s eta 0:00:01
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3160 sha256=91986cf2079fb52cee8bb49fd458fc6653f26be60a9036c38b5cb921c3e785f2
  Stored in directory: /home/rpande7/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=54e8e584551ccdec1b9f7752e517792624c777faea2f3a8f6335f2d0dbc040e3
  Stored in directory: /home/rpande7/.cache/pip/wheels/50/da/35/fe2cfb3bc47822299f5e124a599d56f00b30ec0b328db16b9f
  Created wheel for contextvars: filename=contextvars-2.4-py3-none-any.whl size=7665 sha256=0fcc68dcc495d8c33bb8a8bb4a351a4d50f8df91837e02f9ac765dba7a8c696f
  Stored in directory: /home/rpande7/.cache/pip/wheels/41/11/53/911724983aa48deb94792432e14e518447212dd6c5477d49d3
Successfully built idna-ssl gpustat contextvars
ERROR: pip's dependency resolver does not currently take into account all the

In [ ]:
from utils import *
#import pandas as pd
import modin.pandas as pd

encounters_drop_list=["Unnamed: 2","# Encounters"]
patient_drop_list=['Patient Name','D/C Disp','Unnamed: 9','Discharge Day','Discharge Timestamp','Reason for Visit','Metrics', '# Encs', '# Encs.1', '# Encs.2','# Encs.3', '# Encs.4']
ICD_drop_list=['ICD-10 Diagnosis DESC','Patient Name','Patient ID']
medication_drop_list=['Unnamed: 1','Metrics']
ICD_file_list=['ICD10s_2018_2019.csv','ICD10s_2016_2018.csv','ICD10s_2013_2016.csv']

In [13]:
encounters_df=pd.read_csv("/labs/banerjeelab/Central_line/Data/Encounters.csv")
encounters_df=drop_columns(encounters_drop_list,encounters_df)
encounters_df=encounters_df.dropna()
patients_df=pd.read_csv("/labs/banerjeelab/Central_line/Data/Patient List.csv")
patients_df=drop_columns(patient_drop_list,patients_df)
patients_df=patients_df.dropna()
base_df=concatenate_dfs(encounters_df,patients_df,'Encounter','Encounter')

Columns dropped


/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,8,10,15,16,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Columns dropped
Columns concatenated


In [14]:
base_df['Date of Birth'] = pd.to_datetime(base_df['Date of Birth'], format="%m/%d/%Y")
base_df['Service Day']=pd.to_datetime(base_df['Service Day'], format="%m/%d/%Y")
base_df['Admit Timestamp']= pd.to_datetime(base_df['Admit Timestamp'],errors='coerce', format="%m/%d/%Y %H:%M:%S")
base_df['Age']=abs(base_df['Date of Birth']-base_df['Admit Timestamp'])
base_df['Age']=round(base_df['Age'].dt.days / 365,1)
base_df['EMPI']=base_df['EMPI'].astype(int)
base_df['EMPI']=base_df['EMPI'].astype(str)
base_df=drop_columns(['Date of Birth'],base_df)
base_df=base_df.rename(columns={'Service Day':'Procedure_time','Patient ID':'Patient'})

Columns dropped


In [15]:
base_df[base_df.isna().any(axis=1)]

,Encounter,CPT Procedure,Procedure_time,MRN,Patient,EMPI,Gender,Race,Ethnic Group,Admit Timestamp,Age


## Demographics

In [5]:
print(len(base_df.Patient.unique()))
gender= base_df.drop_duplicates(subset = ["Patient"])
print(len(gender))
print(gender['Gender'].value_counts())
print(gender['Age'].mean(),gender['Age'].std())

29833
29833
Female    14938
Male      14895
Name: Gender, dtype: int64
57.79274628766802 17.25078988391414


## ICD-CODE

In [6]:
ICD_df=pd.DataFrame()
for i in ICD_file_list:
    df=pd.read_csv("/labs/banerjeelab/Central_line/New_data/"+i)
    df=preprocess_dropChars(-3,'ICD-10 Diagnosis Code',df)
    df=df.rename(columns={'ICD-10 Diagnosis Code':'ICD_code','Patient EMPI Nbr':'EMPI'})
    ICD=ICD_mapping('ICD_code',df)
    ICD['Service Day']=pd.to_datetime(ICD['Service Day'], format="%m/%d/%Y")
    ICD=drop_columns(ICD_drop_list,ICD)
    ICD_df=ICD_df.append(ICD)
ICD_df['EMPI']=ICD_df['EMPI'].astype(str)
ICD_df=ICD_df.dropna()
ICD_df['ICD_group']=ICD_df['ICD_group'].astype(int)

-3 Characters dropped
Columns dropped
-3 Characters dropped
Columns dropped
-3 Characters dropped
Columns dropped


In [7]:
ICD_df[ICD_df['ICD_group'].isna()]

,EMPI,Service Day,ICD_code,ICD_group


In [13]:
print(len(base_df))

53910


In [11]:
import datetime
import time
from math import exp
column_names=base_df.columns.tolist()+['I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13','I14','I15','I16','I17','I18','I19','I20','I21','I22','Y']
op2 = pd.DataFrame(columns = column_names)
count=0
start=time.time()
for index,row in base_df.iterrows():
    temp_row=(row.to_dict())
    start_date=row['Procedure_time']-datetime.timedelta(days=2)#row['Admit Timestamp']
    end_date=row['Procedure_time']
    EMPI_val=row['EMPI']
    df=ICD_df[ICD_df['EMPI']==EMPI_val].copy()
    if df.empty==False:
        while end_date-start_date>=datetime.timedelta(days=1):
            df1=df[df['Service Day']<=start_date].copy()
            df1['diff']=(start_date-df1['Service Day'])
            df1['diff']=df1['diff'].dt.days
            df1['weight']=np.exp(-np.power(df1['diff'],2))
            for i in range(1,23): 
                df2=df1[df1['ICD_group']==i].copy()
                temp_row['I'+str(i)]=df2['weight'].sum()
                df2=df2[0:0].copy()
            if(end_date-start_date==datetime.timedelta(days=1)):
                temp_row['Y']=1
            else:
                temp_row['Y']=0
            op2=op2.append(temp_row,ignore_index=True)
            start_date=start_date+datetime.timedelta(days=1)
            df1=df1[0:0].copy()
    df=df[0:0].copy()
    count+=1
    if(count==100):
        print(index)
        print(time.time()-start)
        break

99
74.0119378566742


In [11]:
import datetime
from math import exp
column_names=base_df.columns.tolist()+['I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13','I14','I15','I16','I17','I18','I19','I20','I21','I22','Y']
op2 = pd.DataFrame(columns = column_names)
for index,row in base_df.iterrows():
    start=time.time()
    temp_row=(row.to_dict())
    start_date=row['Admit Timestamp']
    end_date=row['Procedure_time']
    EMPI_val=row['EMPI']
    df=ICD_df[ICD_df['EMPI']==EMPI_val]
    df1=df[df['Service Day']<=start_date]
    df1['diff']=(start_date-df1['Service Day'])
    df1['diff']=df1['diff'].dt.days
    if df.empty==False:
        while end_date-start_date>=datetime.timedelta(days=1):
            df1['weight']=np.exp(-np.power(df1['diff'],2))
            for i in range(1,23): 
                df2=df1[df1['ICD_group']==i]
                temp_row['I'+str(i)]=df2['weight'].sum()
            if(end_date-start_date==datetime.timedelta(days=1)):
                temp_row['Y']=1
            else:
                temp_row['Y']=0
            op2=op2.append(temp_row,ignore_index=True)
            df1['diff']=df1['diff']+1
            df_temp=(df[(df['Service Day']>start_date) & (df['Service Day']<=start_date+datetime.timedelta(days=1))])
            start_date=start_date+datetime.timedelta(days=1)
            df_temp['diff']=(start_date-df_temp['Service Day'])
            df_temp['diff']=df_temp['diff'].dt.days
            df1.append(df_temp)
        print(time.time()-start)

/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0.5932743549346924
0.6005628108978271
0.5969460010528564


/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

1.3905715942382812


...

/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0.5953817367553711


/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

KeyboardInterrupt: 

In [ ]:
op2

## Medications

In [ ]:
df=pd.read_excel("/labs/banerjeelab/Central_line/Data/All Medications.xlsx")
print(len(df.Patient.unique()))
df = df.fillna(method='ffill', axis=0)
df=drop_columns(medication_drop_list,df)
df=df.rename(columns={'Unnamed: 5':'strength'})
df.head()
#df=preprocess_dropChars(-3,'ICD-10 Diagnosis Code',df)
#df=df.rename(columns={'ICD-10 Diagnosis Code':'ICD_code'})

In [ ]:
drug_df=df['Generic Drug Name'].value_counts()
drug_df.to_excel('/home/rpande7/Desktop/Images/Drug_freq.xlsx')